In [1]:
from huggingface_hub import login, snapshot_download
login(token="---") # token 입력할것
snapshot_download(repo_id="xhaka3456/so100_test_0510", repo_type="dataset", local_dir="/home/dr4r/disk/lerobot_dataset/merge_dataset/so100_test_0510")
snapshot_download(repo_id="xhaka3456/so100_test_0412", repo_type="dataset", local_dir="/home/dr4r/disk/lerobot_dataset/merge_dataset/so100_test_0412")

/home/dr4r/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 156 files: 100%|██████████| 156/156 [00:06<00:00, 25.02it/s]


'/home/dr4r/disk/lerobot_dataset/merge_dataset/so100_test_0412'

In [ ]:
# merge code를 터미널에서 실행
python merge_datasets.py \
  --sources /home/dr4r/disk/lerobot_dataset/merge_dataset/so100_test_0510 /home/dr4r/disk/lerobot_dataset/merge_dataset/so100_test_0412 \
  --output /home/dr4r/disk/lerobot_dataset/merge_dataset/integrated_dataset \
  --max_dim 6 \
  --fps 30

In [4]:
from huggingface_hub import HfApi

api = HfApi()
repo_id = "xhaka3456/Pick_Place"

# 레포지토리 생성

try:
    api.create_repo(
        repo_id=repo_id,
        repo_type="dataset",
        private=False  # 공개 레포지토리로 설정 (필요 시 True로 변경)
    )
    print(f"레포지토리 {repo_id}가 성공적으로 생성되었습니다.")
except Exception as e:
    print(f"레포지토리 생성 중 오류 발생: {e}")
    print(f"레포지토리 {repo_id}가 이미 존재할 수 있습니다. 계속 진행합니다.")
    
# data, videos, meta 폴더 업로드
api.upload_folder(
    repo_id=repo_id,
    folder_path="/home/dr4r/disk/lerobot_dataset/merge_dataset/integrated_dataset/data",
    path_in_repo="data",
    repo_type="dataset"
)
api.upload_folder(
    repo_id=repo_id,
    folder_path="/home/dr4r/disk/lerobot_dataset/merge_dataset/integrated_dataset/videos",
    path_in_repo="videos",
    repo_type="dataset"
)
api.upload_folder(
    repo_id=repo_id,
    folder_path="/home/dr4r/disk/lerobot_dataset/merge_dataset/integrated_dataset/meta",
    path_in_repo="meta",
    repo_type="dataset"
)

# README 파일을 새로 생성하고,
# merge된 후 생성되는 meta/info.json 파일의 정보를 README에 추가할것 

# README.md 추가
readme_path = "/home/dr4r/disk/lerobot_dataset/merge_dataset/integrated_dataset/README.md"
with open(readme_path, "a") as f:
    f.write("# Pick_Place_Integrated\n\n"
            "병합된 LeRobot 데이터셋 (so100_test_0510 + so100_test_0412).\n"
            "포맷: observation.images.cam_high (VideoFrame), observation.state (float32 list), action (float32 list), timestamp (float32)")
api.upload_file(
    path_or_fileobj=readme_path,
    path_in_repo="README.md",
    repo_id=repo_id,
    repo_type="dataset"
)

print(f"모든 파일이 {repo_id}에 성공적으로 업로드되었습니다.")

모든 파일이 xhaka3456/Pick_Place에 성공적으로 업로드되었습니다.


In [2]:
from huggingface_hub import HfApi                             

# tag를 업데이트 안하면 오류 발생하므로 아래와 같이 추가 
# 실제 버전은 readme 또는 info.json 에서 확인할것

hub_api = HfApi()
hub_api.create_tag("xhaka3456/Pick_Place", tag="v2.0", repo_type="dataset")